In [15]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [3]:
df_train = df_train.drop(['Ticket', 'Cabin', 'Name'], axis=1)
df_test = df_test.drop(['Ticket', 'Cabin', 'Name'], axis=1)

## Convert String data to numerical data

In [4]:
df_train['Sex'] = df_train['Sex'].map({"male" : 0, "female" : 1})
df_test['Sex'] = df_test['Sex'].map({"male" : 0, "female" : 1})

In [5]:
df_train['Embarked'] = df_train['Embarked'].map({"S" : 0, "C" : 1, "Q" : 2})
df_test['Embarked'] = df_test['Embarked'].map({"S" : 0, "C" : 1, "Q" : 2})

In [6]:
df_train = df_train.dropna(axis=0)
df_test = df_test.dropna(axis=0)

## Fill missing values

In [7]:
df_test['Fare'].fillna(df_test['Fare'].dropna().median(), inplace=True)
df_train['Fare'].fillna(df_train['Fare'].dropna().median(), inplace=True)

In [8]:
df_combined = [df_test, df_train]

In [9]:
guess_ages = np.zeros((2,3))

for dataset in df_combined:
    for i in range(0, 2):
        for j in range(0, 3):
            guess_df = dataset[(dataset['Sex'] == i) & (dataset['Pclass'] == j+1)] ['Age'].dropna()

            age_guess = guess_df.median()

            # Convert random age float to nearest .5 age
            guess_ages[i,j] = int( age_guess/0.5 + 0.5 ) * 0.5
            
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1), 'Age'] = guess_ages[i,j]

    dataset['Age'] = dataset['Age'].astype(int)

## Seperate the data into train and test data sets    

In [10]:
x_train = df_combined[1].drop(['Survived', 'PassengerId'], axis=1)
y_train = df_combined[1]['Survived']

x_test = df_combined[0].drop('PassengerId', axis=1)

# Fit and train

## Support vector machine

In [16]:
svc = SVC()
svc.fit(x_train, y_train)

Y_pred = svc.predict(x_test)
acc_svc = round(svc.score(x_train, y_train) * 100, 2)
acc_svc

67.7

## KNN

In [17]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(x_train, y_train)
Y_pred = knn.predict(x_test)
acc_knn = round(knn.score(x_train, y_train) * 100, 2)
acc_knn

82.44

## Logistic regression

In [18]:
logreg = LogisticRegression()
logreg.fit(x_train, y_train)
Y_pred = logreg.predict(x_test)
acc_log = round(logreg.score(x_train, y_train) * 100, 2)
acc_log

C:\Users\cercenaz\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


80.34